# Image Colorization

In [ ]:
# Project Outline

# 1 Imports
# 2 Data
# 3 Preprocessing 
# 4 Model 
# 5 Training
# 6 Evaluation
# 7 Visualization

In [1]:
# Imports
import numpy as np

In [ ]:
# Data (perhaps should be gradually read in to help with memory)
l_path = "Data/Raw/gray_scale.npy"
ab_paths = ["Data/Raw/ab/ab1.npy", "Data/Raw/ab/ab2.npy", "Data/Raw/ab/ab3.npy"]

l_data = np.load(l_path)
ab_list = [np.load(path) for path in ab_paths]
ab_data = np.concatenate(ab_list, axis=0)

In [ ]:
# Lightness should be in the range of 0 to 100 for absolute black and absolute white respectively 
print(type(l_data))
print(l_data.shape)
print(l_data.max())
print(l_data.min())

# l_data_normalized = (l_data / 128) - 1



<class 'numpy.ndarray'>
(25000, 224, 224)
255
0


In [ ]:
# AB (Green-Magenta and Blue-Yellow) should be in the range of -128 to 127
print(ab_data.shape)
print(ab_data.max())
print(ab_data.min())

# TLDR: Something is wonky about this data; best guess right now is that it used OpenCV's LAB format
# Might have to convert to normal LAB before normalizing to [-1, 1]

(25000, 224, 224, 2)
226
20
